In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import re

### load food dataset

In [4]:
df_food = pd.read_csv('../data/ingredients v1.csv')
df_food.head()

,id,asins,brand,categories,dateAdded,dateUpdated,ean,features.key,features.value,manufacturer,manufacturerNumber,name,sizes,upc,weight,Unnamed: 15
0,AVphBRHOilAPnD_x0OrE,B00HXST15C,Simon Fischer,"Grocery & Gourmet Food,Food,Grocery",2017-01-07T20:13:17Z,2017-06-30T16:48:02Z,41642026706,Ingredients,"Dried Prunes,Water,Corn Syrup,Sugar,Pectin.",Sokol And Company,33829,Simon Fischer Fruit Bttr Prune Lekvar,NaN,41642026706,10.6 pounds,NaN
1,AVpfNFy1LJeJML434ma2,"B008VT0W8C,B0092F8OJ8",McCormick,"Grocery & Gourmet Food,Food,Grocery",2016-02-05T21:08:01Z,2017-04-12T13:54:04Z,52100018164,Ingredients,"Salt,Sugar,Molasses (Refinery Syrup, Molasses,...","McCormick & Co, Inc",MCLANE500373852,McCORMICK GRILL MATES MOLASSES BACON SEASONING...,NaN,52100018164,3.5 ounces,NaN
2,AVpgT49VLJeJML43MJEz,B00CHTWZ2S,Jolly Time,"Grocery & Gourmet Food,Grocery",2014-02-18T01:51:23Z,2017-08-07T16:32:11Z,70670004141,Ingredients,"Salt, Yellow 5 Lake, Tricalcium Phosphate And ...",Reese's,NaN,Jolly Time Popcorn,NaN,70670004141,1.8 pounds,NaN
3,AVphYgnzLJeJML43aPp2,B002JJYNVW,Ziyad,"Grocery & Gourmet Food,grocery",2014-11-04T11:39:27Z,2017-09-04T05:10:37Z,NaN,Ingredients,Mechanically hulled seasame seeds.Allergy Info...,Ziyad,NaN,Ziyad Tahini Sesame Sauce,NaN,74265001560,1.6 pounds,NaN
4,AVpiS0bOLJeJML43kRsh,B00290W1CY,Fla-Vor-Ice,"Grocery & Gourmet Food,grocery",2014-02-18T02:32:12Z,2017-08-26T01:45:28Z,"723,929,530,040,072,000,000,000",Ingredients,FALSE,Fla-Vor-Ice,NaN,Fla-Vor-Ice Plus Giant Pops,NaN,"72,392,952,335,072,300,000,000",18 pounds,NaN


In [5]:
#
df_food.columns

Index(['id', 'asins', 'brand', 'categories', 'dateAdded', 'dateUpdated', 'ean',
       'features.key', 'features.value', 'manufacturer', 'manufacturerNumber',
       'name', 'sizes', 'upc', 'weight', 'Unnamed: 15'],
      dtype='object')

In [6]:
#delete the colummns which are not required
df_food.drop(columns =['id','asins', 'dateAdded', 'dateUpdated', 'ean','features.key','manufacturer', 'manufacturerNumber', 'sizes', 'upc', 'weight', 'Unnamed: 15' ],inplace=True)

In [7]:
df_food.head()

,brand,categories,features.value,name
0,Simon Fischer,"Grocery & Gourmet Food,Food,Grocery","Dried Prunes,Water,Corn Syrup,Sugar,Pectin.",Simon Fischer Fruit Bttr Prune Lekvar
1,McCormick,"Grocery & Gourmet Food,Food,Grocery","Salt,Sugar,Molasses (Refinery Syrup, Molasses,...",McCORMICK GRILL MATES MOLASSES BACON SEASONING...
2,Jolly Time,"Grocery & Gourmet Food,Grocery","Salt, Yellow 5 Lake, Tricalcium Phosphate And ...",Jolly Time Popcorn
3,Ziyad,"Grocery & Gourmet Food,grocery",Mechanically hulled seasame seeds.Allergy Info...,Ziyad Tahini Sesame Sauce
4,Fla-Vor-Ice,"Grocery & Gourmet Food,grocery",FALSE,Fla-Vor-Ice Plus Giant Pops


In [8]:
df_food.to_csv('../data/final_food_list.csv')

In [9]:
#rename column features.values to ingredients list and name to food_product_name
df_food.rename(columns = {
    'features.value':"ingredients",
    'name':'food_product',
    'categories':'food_category'
                        
               },inplace=True)

In [10]:

df_food = df_food[['brand','food_product','ingredients','food_category']]

In [11]:
df_food.head(20)

,brand,food_product,ingredients,food_category
0,Simon Fischer,Simon Fischer Fruit Bttr Prune Lekvar,"Dried Prunes,Water,Corn Syrup,Sugar,Pectin.","Grocery & Gourmet Food,Food,Grocery"
1,McCormick,McCORMICK GRILL MATES MOLASSES BACON SEASONING...,"Salt,Sugar,Molasses (Refinery Syrup, Molasses,...","Grocery & Gourmet Food,Food,Grocery"
2,Jolly Time,Jolly Time Popcorn,"Salt, Yellow 5 Lake, Tricalcium Phosphate And ...","Grocery & Gourmet Food,Grocery"
3,Ziyad,Ziyad Tahini Sesame Sauce,Mechanically hulled seasame seeds.Allergy Info...,"Grocery & Gourmet Food,grocery"
4,Fla-Vor-Ice,Fla-Vor-Ice Plus Giant Pops,FALSE,"Grocery & Gourmet Food,grocery"
5,Hero,Hero Fruit Sprd Blk Currant-12 Oz -pack of 8,"Red Raspberries,Sugar,Glucose Syrup,Citric Aci...","Food,Other Grocery,Grocery"
6,Simply Asia,Simply Asia Noodle Bowl Mandarin Orange -- 8.5 oz,"Noodles: wheat flour,water,wheat gluten,modifi...","Grocery & Gourmet Food,Grocery"
7,EMERIL S,Italian Bread Crumbs,"Wheat Flour,Soybean Oil,Salt,Dehydrated Garlic...","Food,Fresh Food,Grocery"
8,Back to Nature,BACK TO NATURE CEREAL ORANGE CRUNCH GRAN,"Rolled Oats,Brown Rice Syrup,Evaporated Cane J...","Grocery & Gourmet Food,Food"
9,Nutrament,Nutrament Strawberry Complete Nutrition Drink ...,"Skim Milk,Sugar,Corn Syrup,Canola Oil,High Ole...","Food,Fresh Food,Grocery"


In [12]:
df_food.isna().sum()
df_food.shape

(10000, 4)

In [13]:
df_food.dropna(inplace=True)

In [14]:
df_food.shape

(9085, 4)

### load additive dataset

In [16]:
#load the additives data
df_additives = pd.read_csv('../data/additives_with_more_alternate_names.csv')
df_additives.head()

,ingredient,purpose,health concern,alternative_names
0,Acesulfame potassium,Sweetener,"Cancer, Hormone Disruption, Risks to Pregnant ...","Ace-K, Acesulfame K, E950"
1,Aloe vera,Flavoring,Cancer,"Aloe barbadensis, Aloe leaf extract"
2,"Aspartame (Equal, AminoSweet)",Sweetener,Cancer,"Equal, NutraSweet, E951, AminoSweet"
3,Azodicarbonamide,Coloring,Cancer,NaN
4,Brominated vegetable oil (bvo),Emulsifier,Neurological & Behavioral,NaN


In [17]:
# Combine the 'ingredient' and 'alternative_names' columns into one,
# only adding 'alternative_names' if it is not null
df_additives['all_names'] = df_additives.apply(
    lambda row: [row['ingredient']] + row['alternative_names'].split(', ') 
    if pd.notnull(row['alternative_names']) else [row['ingredient']], axis=1
)

# Check for any null values in the new 'all_names' column
null_values = df_additives['all_names'].isnull().sum()

print(f"Number of null values in 'all_names': {null_values}")




Number of null values in 'all_names': 0


In [18]:
df_additives.head(1)

,ingredient,purpose,health concern,alternative_names,all_names
0,Acesulfame potassium,Sweetener,"Cancer, Hormone Disruption, Risks to Pregnant ...","Ace-K, Acesulfame K, E950","[Acesulfame potassium, Ace-K, Acesulfame K, E950]"


In [19]:
df_additives.isna().sum()


ingredient            0
purpose               0
health concern        1
alternative_names    11
all_names             0
dtype: int64

In [20]:
df_additives.shape

(20, 5)

In [21]:
df_additives.to_csv('../data/additive.csv')

### preprocessing the text

In [23]:
#load medium pipeline
nlp = spacy.load('en_core_web_md')

In [24]:
# preprocess text data: lowercase, remove punctuation, tokenize


def preprocess_ingredients(text):
    try:
        if not isinstance(text, str):
            return []  # Return empty if not a string
        #  lowercase
        text = text.lower()
        # replace parentheses with commas
        text = text.replace('(', ',').replace(')', ',')
        # split by commas and strip whitespace
        tokens = [token.strip() for token in text.split(',') if token.strip()]
        return tokens
    except Exception as e:
        return []  



In [25]:

df_food['processed_ingredients'] = df_food['ingredients'].apply(preprocess_ingredients)


In [26]:
df_food.head()

,brand,food_product,ingredients,food_category,processed_ingredients
0,Simon Fischer,Simon Fischer Fruit Bttr Prune Lekvar,"Dried Prunes,Water,Corn Syrup,Sugar,Pectin.","Grocery & Gourmet Food,Food,Grocery","[dried prunes, water, corn syrup, sugar, pectin.]"
1,McCormick,McCORMICK GRILL MATES MOLASSES BACON SEASONING...,"Salt,Sugar,Molasses (Refinery Syrup, Molasses,...","Grocery & Gourmet Food,Food,Grocery","[salt, sugar, molasses, refinery syrup, molass..."
2,Jolly Time,Jolly Time Popcorn,"Salt, Yellow 5 Lake, Tricalcium Phosphate And ...","Grocery & Gourmet Food,Grocery","[salt, yellow 5 lake, tricalcium phosphate and..."
3,Ziyad,Ziyad Tahini Sesame Sauce,Mechanically hulled seasame seeds.Allergy Info...,"Grocery & Gourmet Food,grocery",[mechanically hulled seasame seeds.allergy inf...
4,Fla-Vor-Ice,Fla-Vor-Ice Plus Giant Pops,FALSE,"Grocery & Gourmet Food,grocery",[false]


In [27]:
UNWANTED_TOKENS = {'coloring', 'leaf', 'extract', 'flavor', 'artificial', 'natural', 'spices', 'including','iv'}

def preprocess_additives(text):
    try:
        # Convert the string representation of a list into an actual list
        token_list = eval(text) if isinstance(text, str) else text
        if not isinstance(token_list, list):
            return []  # Return empty if not a list
        # Process each token: lowercase and strip unwanted characters
        processed_tokens = []
        for token in token_list:
            # Convert to lowercase and split by parentheses and commas
            token = token.lower()
            token = token.replace('(', ',').replace(')', ',')
            split_tokens = [t.strip() for t in token.split(',') if t.strip() and t.strip() not in UNWANTED_TOKENS]
            processed_tokens.extend(split_tokens)
        return processed_tokens
    except Exception as e:
        return []  # Return empty list if processing fails

# Apply the revised preprocessing function
df_additives['preprocessed_all_names'] = df_additives['all_names'].apply(preprocess_additives)



In [28]:
df_additives.head()

,ingredient,purpose,health concern,alternative_names,all_names,preprocessed_all_names
0,Acesulfame potassium,Sweetener,"Cancer, Hormone Disruption, Risks to Pregnant ...","Ace-K, Acesulfame K, E950","[Acesulfame potassium, Ace-K, Acesulfame K, E950]","[acesulfame potassium, ace-k, acesulfame k, e950]"
1,Aloe vera,Flavoring,Cancer,"Aloe barbadensis, Aloe leaf extract","[Aloe vera, Aloe barbadensis, Aloe leaf extract]","[aloe vera, aloe barbadensis, aloe leaf extract]"
2,"Aspartame (Equal, AminoSweet)",Sweetener,Cancer,"Equal, NutraSweet, E951, AminoSweet","[Aspartame (Equal, AminoSweet), Equal, NutraSw...","[aspartame, equal, aminosweet, equal, nutraswe..."
3,Azodicarbonamide,Coloring,Cancer,NaN,[Azodicarbonamide],[azodicarbonamide]
4,Brominated vegetable oil (bvo),Emulsifier,Neurological & Behavioral,NaN,[Brominated vegetable oil (bvo)],"[brominated vegetable oil, bvo]"


In [29]:
additives = df_additives['preprocessed_all_names']
additives

0     [acesulfame potassium, ace-k, acesulfame k, e950]
1      [aloe vera, aloe barbadensis, aloe leaf extract]
2     [aspartame, equal, aminosweet, equal, nutraswe...
3                                    [azodicarbonamide]
4                       [brominated vegetable oil, bvo]
5     [butylated hydroxyanisole, bha, bha, e320, but...
6                                    [caramel coloring]
7                                           [cyclamate]
8                                       [ginkgo biloba]
9                                      [olestra, olean]
10    [potassium bromate, e924, bromated flour impro...
11                [potassium iodate, e202, sorbistat-k]
12                                     [propyl gallate]
13                                          [saccharin]
14    [sodium nitrate, nitrite, e251, chile saltpete...
15    [sucralose, splenda, splenda, e955, trichloros...
16                                [synthetic food dyes]
17                       [tbhq, tert-butylhydroq

In [30]:
df_food.head()

,brand,food_product,ingredients,food_category,processed_ingredients
0,Simon Fischer,Simon Fischer Fruit Bttr Prune Lekvar,"Dried Prunes,Water,Corn Syrup,Sugar,Pectin.","Grocery & Gourmet Food,Food,Grocery","[dried prunes, water, corn syrup, sugar, pectin.]"
1,McCormick,McCORMICK GRILL MATES MOLASSES BACON SEASONING...,"Salt,Sugar,Molasses (Refinery Syrup, Molasses,...","Grocery & Gourmet Food,Food,Grocery","[salt, sugar, molasses, refinery syrup, molass..."
2,Jolly Time,Jolly Time Popcorn,"Salt, Yellow 5 Lake, Tricalcium Phosphate And ...","Grocery & Gourmet Food,Grocery","[salt, yellow 5 lake, tricalcium phosphate and..."
3,Ziyad,Ziyad Tahini Sesame Sauce,Mechanically hulled seasame seeds.Allergy Info...,"Grocery & Gourmet Food,grocery",[mechanically hulled seasame seeds.allergy inf...
4,Fla-Vor-Ice,Fla-Vor-Ice Plus Giant Pops,FALSE,"Grocery & Gourmet Food,grocery",[false]


In [31]:
# set of processed additive tokens for labeling
unhealthy_additives = set(
    token
    for tokens in df_additives['preprocessed_all_names']
    for token in tokens
)
print(unhealthy_additives)
# labeling function 
def label_food(ingredients):
    if not isinstance(ingredients, list):
        return "Healthy"  # Default to Healthy if no valid ingredients
    # Check if any ingredient matches the set of unhealthy additives
    for ingredient in ingredients:
        if ingredient in unhealthy_additives:
            return "Not Healthy"
    return "Healthy"

# label the data 
df_food['health_label'] = df_food['processed_ingredients'].apply(label_food)



{'oxide', 'splenda', 'butylated hydroxyanisole', 'trans fat', 'partially hydrogenated vegetable oil', 'e955', 'nitrite', 'sucralose', 'tert-butylhydroquinone', 'ginkgo biloba', 'synthetic food dyes', 'tio2', 'ace-k', 'bha', 'e320', 'aspartame', 'e171', 'chile saltpeter', 'e202', 'sodium nitrate', 'aminosweet', 'e924', 'kbro3', 'e251', 'azodicarbonamide', 'bvo', 'aloe barbadensis', 'potassium iodate', 'aloe leaf extract', 'titanium', 'acesulfame potassium', 'brominated vegetable oil', 'acesulfame k', 'trichlorosucrose', 'butylhydroxyanisole', 'aloe vera', 'bromated flour improver', 'sorbistat-k', 'potassium bromate', 'propyl gallate', 'e950', 'olestra', 'nitrate of soda', 'e951', 'olean', 'saccharin', 'cyclamate', 'tbhq', 'caramel coloring', 'equal', 'nutrasweet', 'titanium dioxide'}


In [32]:

df_food_unhealthy = df_food[df_food['health_label'] == 'Not Healthy']
df_food_unhealthy

,brand,food_product,ingredients,food_category,processed_ingredients,health_label
1,McCormick,McCORMICK GRILL MATES MOLASSES BACON SEASONING...,"Salt,Sugar,Molasses (Refinery Syrup, Molasses,...","Grocery & Gourmet Food,Food,Grocery","[salt, sugar, molasses, refinery syrup, molass...",Not Healthy
54,Dickinson's,Preserv Sf Blckbrry -pack of 6,"Water (Ingredients Not in Regular Preserves),B...","Food,Grocery","[water, ingredients not in regular preserves, ...",Not Healthy
92,Life Savers,"Life Savers Sugar Free Wild Cherry Hard Candy,...","Isomalt,Citric Acid,Natural And Artificial Fla...","Food,Grocery","[isomalt, citric acid, natural and artificial ...",Not Healthy
106,Smucker's,Smucker&#39;s: Boysenberry Sugar Free Preserve...,"Water**,Boysenberries+,Polydextrose**,Maltodex...","Food,Grocery","[water**, boysenberries+, polydextrose**, malt...",Not Healthy
129,HERSHEY'S,"Hershey's Whatchamacallit King Size Candy Bar,...","Sugar,High Fructose Corn Syrup,Crisp Rice,Part...","Food,Grocery","[sugar, high fructose corn syrup, crisp rice, ...",Not Healthy
...,...,...,...,...,...,...
9820,Maple Grove Farms,Maple Grove FarmsÂ® Sugar Free Low Calorie Map...,"Water,Sorbitol,Cellulose Gum,Natural and Artif...","Food,Meal Solutions, Grains & Pasta,Canned Goo...","[water, sorbitol, cellulose gum, natural and a...",Not Healthy
9852,Jolly Time,Jolly Time Butter,"Popcorn,Corn Oil,Palm Oil,Sodium Chloride,Natu...","Food,Packaged Foods,Snacks,Popcorn,Microwave P...","[popcorn, corn oil, palm oil, sodium chloride,...",Not Healthy
9867,Maggi,MAGGI Chicken Flavor Noodle Soup Mix 2.11 oz. ...,"Wheat Flour,Salt,Cornstarch,Sugar,Monosodium G...","Food,Meal Solutions, Grains & Pasta,Canned Goo...","[wheat flour, salt, cornstarch, sugar, monosod...",Not Healthy
9893,Allegro,Allegro Original Marinade 12. 7 Oz -pack of 6,"Soy Sauce (Water, Salt, Hydrollyzed Soy Prnoti...","Food,Sauces,#20154 in,#3052 in,#331 in,grocery","[soy sauce, water, salt, hydrollyzed soy prnot...",Not Healthy


In [33]:
#verify unhealthy labels 

In [34]:
# verify if the label is correct
df_food['is_correct'] = df_food.apply(
    lambda row: (
        'Correct' if (
            (any(ingredient in unhealthy_additives for ingredient in row['processed_ingredients'])
             and row['health_label'] == 'Not Healthy') or
            (not any(ingredient in unhealthy_additives for ingredient in row['processed_ingredients'])
             and row['health_label'] == 'Healthy')
        ) else 'Incorrect'
    ),
    axis=1
)


incorrect_labels = df_food[df_food['is_correct'] == 'Incorrect']
incorrect_labels

,brand,food_product,ingredients,food_category,processed_ingredients,health_label,is_correct


In [84]:

w = df_food[df_food['food_product'] == 'Jolly Time Butter']
w

,brand,food_product,ingredients,food_category,processed_ingredients,health_label,is_correct
9852,Jolly Time,Jolly Time Butter,"Popcorn,Corn Oil,Palm Oil,Sodium Chloride,Natu...","Food,Packaged Foods,Snacks,Popcorn,Microwave P...","[popcorn, corn oil, palm oil, sodium chloride,...",Not Healthy,Correct
